In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

import qnmfits
import analytic_fit
import utils
import corner

from cmcrameri import cm

plt.rcParams.update(utils.rcparams)

In [ ]:
rng = np.random.default_rng(12345)

In [ ]:
t0_M = pd.read_csv('../data/t0_data_30_to_100.csv', index_col=0)

ID = 1
N = 20
T = 100

t0 = t0_M[str(ID)].values[N]

In [ ]:
injection_info, injection_C, injection_omega = utils.injection(
    ID = 1,
    N = N,
    tref=t0,
    returnC=True
)
injection = injection_info['sim']

In [ ]:
N_min = 0
N_max = 20

N_list = list(range(N_min,N_max+1))

# Create a custom colormap
color_list = [cm.roma(N/N_max) for N in N_list]
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    None, color_list, N=len(color_list)
    )
norm = mpl.colors.Normalize(vmin=N_min-0.5, vmax=N_max+0.5)

In [ ]:
mode_list = utils.get_mode_list(N_max)

mm_times = injection.times[(injection.times > -30) & (injection.times < 100)]

mm_lists = []
for i, modes in enumerate(mode_list):
    mm_list = qnmfits.mismatch_t0_array(
        injection.times,
        injection.h[2, 2],
        modes,
        Mf=injection.Mf,
        chif=injection.chif_mag,
        t0_array=mm_times,
        t0_method='closest'
    )
    mm_lists.append(mm_list)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
ax.set_yscale('log')
ax.grid(alpha=0.3)

cbar = fig.colorbar(
    mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
    ax=ax,
    location='right',
    ticks=range(N_min, N_max+1),
)

cbar.set_label(r'Number of overtones, $N$', labelpad=7)

bounds = [n+0.5 for n in range(N_min, N_max)]
for bound in bounds:
    cbar.ax.axhline(bound, color='k', lw=0.5, alpha=0.2)

for n in N_list:
    
    ax.plot(
        mm_times,
        mm_lists[n],
        color=cmap(n/N_max),
        zorder=1.1,
        alpha=0.5,
        lw=1
        )

ax.set_xlim(-25, 60)
ax.set_ylim(1e-10, 1)

ax.set_xlabel(r'Fit time, $t_0 - t_\mathrm{peak}^{h_{22}}\,[M]$')
ax.set_ylabel(r'Mismatch, $\mathcal{M}$');

In [ ]:
sigma = 1e-5
noise = rng.normal(0, sigma, size=injection.h[2,2].shape) + \
    1j*rng.normal(0, sigma, size=injection.h[2,2].shape)

mm_lists = []
for i, modes in enumerate(mode_list):
    mm_list = qnmfits.mismatch_t0_array(
        injection.times,
        injection.h[2, 2] + noise,
        modes,
        Mf=injection.Mf,
        chif=injection.chif_mag,
        t0_array=mm_times,
        t0_method='closest'
    )
    mm_lists.append(mm_list)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
ax.set_yscale('log')
ax.grid(alpha=0.3)

cbar = fig.colorbar(
    mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
    ax=ax,
    location='right',
    ticks=range(N_min, N_max+1),
)

cbar.set_label(r'Number of overtones, $N$', labelpad=7)

bounds = [n+0.5 for n in range(N_min, N_max)]
for bound in bounds:
    cbar.ax.axhline(bound, color='k', lw=0.5, alpha=0.2)

for n in N_list:
    
    ax.plot(
        mm_times,
        mm_lists[n],
        color=cmap(n/N_max),
        zorder=1.1,
        alpha=0.5,
        lw=1
        )

ax.set_xlim(-25, 60)
ax.set_ylim(1e-10, 1)

ax.set_xlabel(r'Fit time, $t_0 - t_\mathrm{peak}^{h_{22}}\,[M]$')
ax.set_ylabel(r'Mismatch, $\mathcal{M}$');